In [24]:
pip install -U torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu121

Looking in indexes: https://download.pytorch.org/whl/cu121
Note: you may need to restart the kernel to use updated packages.


ERROR: Could not find a version that satisfies the requirement torchaudio (from versions: none)

[notice] A new release of pip is available: 25.1.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip
ERROR: No matching distribution found for torchaudio


In [16]:
pip install -q torch torchvision transformers datasets evaluate pillow tqdm hf_xet ipywidgets ipykernel jupyter scikit-learn

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.1.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


pip install transformers[torch]pip install transformers[torch]Loading RVL-CDIP DAtaset from HF

In [1]:
import os
os.environ["HF_HUB_DISABLE_PROGRESS_BARS"] = "1"
os.environ["HF_DATASETS_DISABLE_PROGRESS_BARS"] = "1"
os.environ["TQDM_DISABLE"] = "0"
os.environ["TQDM_NOTEBOOK"] = "0"

from datasets import load_dataset

dataset = load_dataset("chainyo/rvl-cdip")
print(dataset)

# Get label names
labels = dataset["train"].features["label"].names
print("Detected classes:", labels)
print(f"Number of classes: {len(labels)}")


DatasetDict({
    train: Dataset({
        features: ['image', 'label'],
        num_rows: 319999
    })
    test: Dataset({
        features: ['image', 'label'],
        num_rows: 40000
    })
    val: Dataset({
        features: ['image', 'label'],
        num_rows: 40000
    })
})
Detected classes: ['advertisement', 'budget', 'email', 'file folder', 'form', 'handwritten', 'invoice', 'letter', 'memo', 'news article', 'presentation', 'questionnaire', 'resume', 'scientific publication', 'scientific report', 'specification']
Number of classes: 16


 Load model + processor

In [3]:
from transformers import AutoProcessor, LayoutLMv3ForSequenceClassification

model_name = "microsoft/layoutlmv3-base"

processor = AutoProcessor.from_pretrained(model_name)
model = LayoutLMv3ForSequenceClassification.from_pretrained(
    model_name, num_labels=len(labels)
)


Some weights of LayoutLMv3ForSequenceClassification were not initialized from the model checkpoint at microsoft/layoutlmv3-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Preprocess images

In [4]:
def preprocess(example):
    encoding = processor(images=example["image"], return_tensors="pt")
    return {k: v.squeeze() for k, v in encoding.items()}

dataset = dataset.with_transform(preprocess)

Metrics

In [7]:
import numpy as np
import evaluate

accuracy = evaluate.load("accuracy")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    preds = np.argmax(logits, axis=-1)
    return accuracy.compute(predictions=preds, references=labels)


In [23]:
import torch
print("Torch version:", torch.__version__)
print("Torch location:", torch.__file__)
print("CUDA available:", torch.cuda.is_available())


Torch version: 2.9.0+cpu
Torch location: D:\Projects\DSPRO1\document-classifier\.venv\Lib\site-packages\torch\__init__.py
CUDA available: False


Training setup

In [25]:
import transformers
from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(
    output_dir="./rvlcdip-results",
    eval_strategy="epoch",  # Changed from evaluation_strategy
    save_strategy="epoch",
    logging_strategy="steps",
    logging_steps=50,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    num_train_epochs=3,
    learning_rate=5e-5,
    load_best_model_at_end=True,
    metric_for_best_model="accuracy",
    fp16=True,  # set to False if you're on CPU
    report_to="none",  # disable wandb/hub logging
)

PyTorch: setting up devices


ImportError: Using the `Trainer` with `PyTorch` requires `accelerate>=0.26.0`: Please run `pip install transformers[torch]` or `pip install 'accelerate>=0.26.0'`

Trainer

In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=dataset["train"],
    eval_dataset=dataset["validation"],
    compute_metrics=compute_metrics,
)

trainer.train()


Evaluate final model

In [ ]:
metrics = trainer.evaluate(dataset["test"])
print(metrics)

Save model + processor

In [ ]:
save_path = "./layoutlmv3-rvlcdip-model"
model.save_pretrained(save_path)
processor.save_pretrained(save_path)
print(f"Model saved to {save_path}")